In [ ]:
%matplotlib notebook
import numpy as numpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
DEATH_BY_COV = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
CASES_COV = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [ ]:
def data_preparation(url):
    """
    Function to prepare the data to be analyzed.
    """
    data_df = pd.read_csv(url)
    colums_exclud = data_df.columns[[0, 2, 3]]
    data_df.drop(colums_exclud,
                 axis=1,
                 inplace=True)
    data_df = data_df.set_index("Country/Region")
    data_df = data_df.groupby(level=0).sum()
    return data_df


In [ ]:
def data_country(cases_table, death_table, country):
    """
    Function to create a matrix with data of confirmed cases of COVID-19 and death by COVID-19.
    The first column is data-times, the second column is confirmed cases of COVID-19, and the
    third column is deaths by COVID-19.
    PARAMETERS
    ----------
    data_table: DataFrame with cases
    death_table: DataFrame with deaths
    country: Country to analize
    """
    data_cases = cases_table.loc[country]
    data_death = death_table.loc[country]
    data_array = np.c_[data_cases.index,
                       data_cases.values,
                       data_death.values]
    return data_array

In [ ]:
cases_df = data_preparation(url=CASES_COV)
death_df = data_preparation(url=DEATH_BY_COV)

In [ ]:
# def country_plot(country, subplot=True, save=False)
# sns.set_style("dark")
country = "Brazil"


labels = "Cases Deaths".split(" ")
colors = "orange red".split(" ")
linestyle = "-. --".split(" ")
linewidth = 0.8
step_x = 20
step_y = 15
size = 12
labely = "Confirmed COVID-19 cases"
labely2 = "Deaths reported by COVID-19"


data_df = data_country(cases_table=cases_df, death_table=death_df, country=country)
date_time = np.array(list(map(lambda x: datetime.strptime(x, "%m/%d/%y"),
                              data_df[:, 0])))
cases = data_df[:, 1]
deaths = data_df[:, 2]

fig, ax = plt.subplots(figsize=(10, 6))
ax_death = ax.twinx()

ax.plot(date_time,
        cases,
        label=f"COVID-19 cases",
        ls=linestyle[0],
        linewidth=linewidth,
        color=colors[0])
ax.plot(np.nan,
        label=f"COVID-19 deaths",
        linewidth=linewidth,
        ls=linestyle[1],
        color=colors[1])

ax_death.plot(date_time,
              deaths,
              label=f"COVID-19 deaths",
              linewidth=linewidth,
              ls=linestyle[1],
              color=colors[1])

data_ticks = date_time[np.arange(0,
                                len(date_time),
                                step_x)]
ax.set_xticks(data_ticks)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%d'))
ax.xaxis.set_tick_params(rotation=0,
                         labelsize=size-3)


cases_tick = np.linspace(0,
                         cases[-1],
                         step_y,
                         dtype=int)
ax.set_yticks(cases_tick)
ax.yaxis.set_tick_params(labelsize=size-3)
ax.set_ylabel(labely,
              size=size)

death_tick = np.linspace(0,
                         deaths[-1],
                         step_y,
                         dtype=int)
ax_death.set_yticks(death_tick)
ax_death.yaxis.set_tick_params(labelsize=size-3)
ax_death.set_ylabel(labely2,
                    size=size)
ax.legend(loc=4,
          fontsize=size-4,
          frameon=False)
